In [3]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# fetching LogisticRegression and dataset rom sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

digits = load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=0)
logReg = LogisticRegression()

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)  

logReg.fit(x_train,y_train)

LogisticRegression()

In [17]:
predictions = logReg.predict(x_test)
score = logReg.score(x_test, y_test)
print(score)

0.7305555555555555
